In [177]:
# importing required pakcages
import pandas as pd
from tqdm import tqdm
import yaml

In [178]:
def read_yaml(file_path):
    with open(file_path, "r") as f:
        return yaml.safe_load(f)

ds = read_yaml(file_path = "configs.yaml")

In [179]:
ds_config = "AmazonToy"

In [180]:
dataset = pd.read_csv(f"datasets/{ds[ds_config]['ds_name']}/ratings_data.txt", sep=ds[ds_config]['ds_sep'], names=ds[ds_config]['ds_names'])

In [181]:
dataset.head()

,uid,iid,rating,timestamp
0,A1VXOAVRGKGEAK,0439893577,5,1390953600
1,A8R62G708TSCM,0439893577,4,1395964800
2,A21KH420DK0ICA,0439893577,5,1359331200
3,AR29QK6HPFYZ4,0439893577,5,1391817600
4,ACCH8EOML6FN5,0439893577,4,1399248000


In [182]:
# what columns or attributes the dataset is included?
dataset.columns

Index(['uid', 'iid', 'rating', 'timestamp'], dtype='object')

In [183]:
# statistics on explicit dataset
print("Original dataset statistics: ")
print(f"> No. of users: {len(dataset['uid'].unique())}")
print(f"> No. of Items: {len(dataset['iid'].unique())}")
print(f"> No. of Interaction: {dataset.shape[0]}")

Original dataset statistics: 
> No. of users: 19412
> No. of Items: 11924
> No. of Interaction: 167597


In [184]:
# checking the number of times in which a rating happened - (0: implicit, 1-10: explicit)
dataset['rating'].value_counts()

5    102790
4     37445
3     16357
2      6298
1      4707
Name: rating, dtype: int64

In [185]:
# This method return a dataframe in which the specified values are removed from a specific column. We can use it to generate the implicit/explicit dataset.
# To do this, we can remove the 0 or 1-10 values from the `Book-Rating` column.
def filter_rows_by_values(df, col, values):
    return df[~df[col].isin(values)]

In [186]:
# To remove the users with fewer than 5 interaction we first count the number of interactino per user and add a new column (`Count`) in the dataframe.
# This column shows the number of interaction per user in the dataset
users_counts = dataset['uid'].value_counts()
users_counts = users_counts.to_dict() #converts to dictionary
dataset['count'] = dataset['uid'].map(users_counts)

### Iteratively remove users and items with fewer than `ds_rate` interactions

In [187]:
user_interaction, item_interaction = 1, 1

while user_interaction != 0 or item_interaction != 0:
    print(f"The current number of user and item with < {ds[ds_config]['ds_core']} interactions: ")
    # user side fewer than ds['ds_core'] cheking
    uid_value_counts = dataset['uid'].value_counts()
    user_interaction = uid_value_counts[uid_value_counts < ds[ds_config]['ds_core']].count()
    print(f"No. of users < {ds[ds_config]['ds_core']} ineractions: {user_interaction}")

    users_counts = dataset['uid'].value_counts()
    users_counts = users_counts.to_dict() #converts to dictionary
    dataset['count'] = dataset['uid'].map(users_counts)

    dataset = filter_rows_by_values(dataset, "count", list(range(ds[ds_config]['ds_core'])))

    # item side fewer than ds_rate cheking
    bid_value_counts = dataset['iid'].value_counts()
    item_interaction = bid_value_counts[bid_value_counts < ds[ds_config]['ds_core']].count()
    print(f"No. of items < {ds[ds_config]['ds_core']} ineractions: {item_interaction}")

    items_counts = dataset['iid'].value_counts()
    items_counts = items_counts.to_dict() #converts to dictionary
    dataset['count'] = dataset['iid'].map(items_counts)

    dataset = filter_rows_by_values(dataset, "count", list(range(ds[ds_config]['ds_core'])))

The current number of user and item with < 7 interactions: 
No. of users < 7 ineractions: 10401
No. of items < 7 ineractions: 6910
The current number of user and item with < 7 interactions: 
No. of users < 7 ineractions: 3924
No. of items < 7 ineractions: 1690
The current number of user and item with < 7 interactions: 
No. of users < 7 ineractions: 1304
No. of items < 7 ineractions: 612
The current number of user and item with < 7 interactions: 
No. of users < 7 ineractions: 578
No. of items < 7 ineractions: 333
The current number of user and item with < 7 interactions: 
No. of users < 7 ineractions: 343
No. of items < 7 ineractions: 190
The current number of user and item with < 7 interactions: 
No. of users < 7 ineractions: 193
No. of items < 7 ineractions: 102
The current number of user and item with < 7 interactions: 
No. of users < 7 ineractions: 98
No. of items < 7 ineractions: 59
The current number of user and item with < 7 interactions: 
No. of users < 7 ineractions: 64
No. of 

In [188]:
# statistics on 5 rate explicit dataset (after pre-processing)
print(f"No. of users: {len(dataset['uid'].unique())}")
print(f"No. of Items: {len(dataset['iid'].unique())}")
print(f"No. of Interaction: {dataset.shape[0]}")

No. of users: 2170
No. of Items: 1733
No. of Interaction: 32852


In [189]:
# Before we save the preprocessed explicit dataset (5Rate) we first remove the added column which is `Count`
del dataset['count']
if 'timestamp' in ds[ds_config]['ds_names']:
    del dataset['timestamp']

### User and Item ID Mapping

In [190]:
uid_to_index = dict()
iid_to_index = dict()

last_user_id = 0
last_item_id = 0

for eachline in tqdm(dataset.iterrows()):
  # add a new user id with an index
  if eachline[1][0] not in uid_to_index.keys():
    uid_to_index[eachline[1][0]] = last_user_id
    last_user_id += 1
  # add a new book id with an index
  if eachline[1][1] not in iid_to_index.keys():
    iid_to_index[eachline[1][1]] = last_item_id
    last_item_id += 1

32852it [00:04, 7674.75it/s]


In [191]:
# write the file with new mapped indices into a txt file
# mapped_dataset = open(f"datasets/{ds['ds_name']}/{ds['ds_name']}_data.csv", 'w')
mapped_dataset_txt = open(f"datasets/{ds[ds_config]['ds_name']}/{ds[ds_config]['ds_name']}_data.txt", 'w')
# mapped_dataset.write('uid' + ',' + 'bid' + ',' + 'rating' + '\n')

for eachline in tqdm(dataset.iterrows()):
    # mapped_dataset.write(str(uid_to_index[eachline[1][0]]) + "," + str(iid_to_index[eachline[1][1]]) + "," + str(eachline[1][2]) + "\n")
    mapped_dataset_txt.write(str(uid_to_index[eachline[1][0]]) + "," + str(iid_to_index[eachline[1][1]]) + "," + str(eachline[1][2]) + "\n")
# mapped_dataset.close()
mapped_dataset_txt.close()

32852it [00:05, 5999.91it/s]
